# Matching des identifiants des bureaux de vote du REU et du Ministère de l'Intérieur

## Imports

### Packages

In [1]:
import os
import pandas as pd
import re
from unidecode import unidecode

### Fonctions

In [2]:
def complete_postal_code(x):
    # The result shall be a 5-digits postal code
    x = str(x or "").strip()
    while len(x) > 0 and x[0] in ["-", "o"]:
        x = x[1:]
    nb_digits = min(len(x), 5)
    return ((5 - nb_digits) * "0" + x)[:5]

In [3]:
list_chars = [".", ",", ";", ":", "#", "°", "*", '"', "-", "/", "'", "  ", "BUREAU"]  # Undesirable punctuation

def clean_place_name(x):
    """
    Removes what is between parenthesis or brackets, lowers the string and removes undesirable punctuation
    """
    x_clean = re.sub("[\(\[].*?[\)\]]", "", x)
    for character in list_chars:
        x_clean = x_clean.replace(character, " ")
    x_clean = ''.join([str(int(i)) + " " if i.isnumeric() else i.strip() for i in x_clean.split(" ") if len(i) > 0])
    while len(x_clean) > 0 and x_clean[0] == '0':
        x_clean = x_clean[1:]
    return unidecode(x_clean).strip()

### Données

In [4]:
os.system(f"mc cp s3/projet-adresses-reu/table_bv.parquet table_bv.parquet")

In [5]:
df_bv_originale = pd.read_parquet('table_bv.parquet')
df_bv_originale['nb_adresses_final'] = df_bv_originale['nb_adresses_final'].fillna(0)

In [6]:
df_bv = df_bv_originale.copy()
df_bv.head(3)

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final
0,01001_1,1001_1,01001,1,bureau 1,,Salle des fêtes,01400,abergement clemenciat,448,448.0
1,01002_1,1002_1,01002,1,mairie,1,Place de la Mairie,01640,l abergement de varey,157,143.0
2,01004_1,1004_1,01004,1,b1 espace 1500,,AVENUE LEON BLUM,01500,amberieu en bugey,633,630.0


In [7]:
df_bv["code"] = df_bv["code"].apply(clean_place_name)
df_bv["departement"] = df_bv["code_commune"].apply(lambda x: x[:2])

In [8]:
os.system(f"mc cp s3/projet-adresses-reu/resultats-par-niveau-burvot-t1-france-entiere.xlsx table_miom.xlsx")

In [9]:
df_miom_originale = pd.read_excel('table_miom.xlsx', dtype=str)

In [10]:
df_miom = df_miom_originale.copy()
df_miom

,Code du département,Libellé du département,Code de la circonscription,Libellé de la circonscription,Code de la commune,Libellé de la commune,Code du b.vote,Inscrits,Abstentions,% Abs/Ins,...,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104
0,01,Ain,04,4ème circonscription,001,L'Abergement-Clémenciat,0001,645,108,16.74,...,3,0.47,0.58,12,M,DUPONT-AIGNAN,Nicolas,21,3.26,4.04
1,01,Ain,05,5ème circonscription,002,L'Abergement-de-Varey,0001,213,38,17.84,...,3,1.41,1.75,12,M,DUPONT-AIGNAN,Nicolas,0,0,0
2,01,Ain,05,5ème circonscription,004,Ambérieu-en-Bugey,0001,1129,266,23.56,...,8,0.71,0.95,12,M,DUPONT-AIGNAN,Nicolas,13,1.15,1.55
3,01,Ain,05,5ème circonscription,004,Ambérieu-en-Bugey,0002,1128,265,23.49,...,4,0.35,0.48,12,M,DUPONT-AIGNAN,Nicolas,20,1.77,2.38
4,01,Ain,05,5ème circonscription,004,Ambérieu-en-Bugey,0003,1213,246,20.28,...,3,0.25,0.32,12,M,DUPONT-AIGNAN,Nicolas,19,1.57,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69677,ZZ,Français établis hors de France,06,6ème circonscription,229,Zurich,0001,24868,14101,56.7,...,47,0.19,0.44,12,M,DUPONT-AIGNAN,Nicolas,189,0.76,1.77
69678,ZZ,Français établis hors de France,11,11ème circonscription,231,Taipei,0001,1709,942,55.12,...,10,0.59,1.32,12,M,DUPONT-AIGNAN,Nicolas,10,0.59,1.32
69679,ZZ,Français établis hors de France,11,11ème circonscription,233,Nour-Soultan,0001,117,64,54.7,...,0,0,0,12,M,DUPONT-AIGNAN,Nicolas,1,0.85,1.96
69680,ZZ,Français établis hors de France,02,2ème circonscription,234,Monterrey,0001,713,553,77.56,...,1,0.14,0.63,12,M,DUPONT-AIGNAN,Nicolas,1,0.14,0.63


### Informations du Ministère

In [11]:
table_correspondance_outremer = {
    "ZA":"971",
    "ZB":"972",
    "ZC":"973",
    "ZD":"974",
    "ZS":"975",
    "ZM":"976",
    "ZY":"977",
    "ZT":"978",
    "ZW":"986",
    "ZP":"987"
}            

## Preprocessing des bv MIOM

### Pré-création id_brut

In [12]:
df_miom["code_commune"] = df_miom["Code du département"] + df_miom["Code de la commune"]
df_miom["code"] = df_miom["Code du b.vote"]
df_miom["commune_miom"] = df_miom["Libellé de la commune"]
df_miom["libelle_departement_miom"] = df_miom["Libellé du département"]

In [13]:
df_miom['id_brut_miom'] = \
    df_miom['code_commune'].fillna("") + '_' + df_miom['code'].fillna("")

### Post création id_brut

In [14]:
df_miom["code"] = df_miom["code"].astype(str).str.lower().apply(clean_place_name)

In [15]:
df_miom["Code de la commune"] = df_miom.apply(
    lambda row: row["Code de la commune"][1:] if row["Code du département"] in table_correspondance_outremer.keys() else row["Code de la commune"],
    axis=1
)

df_miom["Code du département"] = df_miom["Code du département"].apply(
    lambda x: table_correspondance_outremer[x] if x in table_correspondance_outremer.keys() else x
)
df_miom["code_commune"] = df_miom["Code du département"] + df_miom["Code de la commune"]
df_miom["departement_miom"] = df_miom["Code du département"]

In [16]:
df_miom['id_clean_miom'] = \
    df_miom['code_commune'].fillna("") + '_' + df_miom['code'].fillna("")

In [17]:
df_bv_miom = df_miom[["id_brut_miom", "id_clean_miom", "code_commune", "code", "commune_miom", "departement_miom", "libelle_departement_miom"]]
df_bv_miom

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
0,01001_0001,01001_1,01001,1,L'Abergement-Clémenciat,01,Ain
1,01002_0001,01002_1,01002,1,L'Abergement-de-Varey,01,Ain
2,01004_0001,01004_1,01004,1,Ambérieu-en-Bugey,01,Ain
3,01004_0002,01004_2,01004,2,Ambérieu-en-Bugey,01,Ain
4,01004_0003,01004_3,01004,3,Ambérieu-en-Bugey,01,Ain
...,...,...,...,...,...,...,...
69677,ZZ229_0001,ZZ229_1,ZZ229,1,Zurich,ZZ,Français établis hors de France
69678,ZZ231_0001,ZZ231_1,ZZ231,1,Taipei,ZZ,Français établis hors de France
69679,ZZ233_0001,ZZ233_1,ZZ233,1,Nour-Soultan,ZZ,Français établis hors de France
69680,ZZ234_0001,ZZ234_1,ZZ234,1,Monterrey,ZZ,Français établis hors de France


In [18]:
df_miom[df_miom["Code du département"].apply(lambda x: not(x.isnumeric()))].groupby(['Code du département']).count()

,Libellé du département,Code de la circonscription,Libellé de la circonscription,Code de la commune,Libellé de la commune,Code du b.vote,Inscrits,Abstentions,% Abs/Ins,Votants,...,Unnamed: 102,Unnamed: 103,Unnamed: 104,code_commune,code,commune_miom,libelle_departement_miom,id_brut_miom,departement_miom,id_clean_miom
Code du département,,,,,,,,,,,,,,,,,,,,,
2A,220,220,220,220,220,220,220,220,220,220,...,220,220,220,220,220,220,220,220,220,220
2B,325,325,325,325,325,325,325,325,325,325,...,325,325,325,325,325,325,325,325,325,325
ZN,297,297,297,297,297,297,297,297,297,297,...,297,297,297,297,297,297,297,297,297,297
ZX,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
ZZ,210,210,210,210,210,210,210,210,210,210,...,210,210,210,210,210,210,210,210,210,210


## Matching

### Y a-t-il des dupliqués ?

In [19]:
duplicates_reu = df_bv[df_bv['id_brut_reu'].duplicated(keep=False)]
duplicates_reu

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement


In [20]:
duplicates_miom = df_bv_miom[df_bv_miom['id_clean_miom'].duplicated(keep=False)]
duplicates_miom

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom


### Jointure

In [21]:
df_inner_matching = pd.merge(df_bv, df_bv_miom,
                           on=[
                               'code', 'code_commune'
                           ],
                           how='inner')

In [22]:
print(len(df_inner_matching))

df_inner_matching.head()

66375


,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement,id_brut_miom,id_clean_miom,commune_miom,departement_miom,libelle_departement_miom
0,01001_1,1001_1,01001,1,bureau 1,,Salle des fêtes,01400,abergement clemenciat,448,448.0,01,01001_0001,01001_1,L'Abergement-Clémenciat,01,Ain
1,01002_1,1002_1,01002,1,mairie,1,Place de la Mairie,01640,l abergement de varey,157,143.0,01,01002_0001,01002_1,L'Abergement-de-Varey,01,Ain
2,01004_1,1004_1,01004,1,b1 espace 1500,,AVENUE LEON BLUM,01500,amberieu en bugey,633,630.0,01,01004_0001,01004_1,Ambérieu-en-Bugey,01,Ain
3,01004_2,1004_2,01004,2,b2 espace 1500,,AVENUE LEON BLUM,01500,amberieu en bugey,640,638.0,01,01004_0002,01004_2,Ambérieu-en-Bugey,01,Ain
4,01004_3,1004_3,01004,3,b3 chateau des echelles,,RUE DES ARENES,01500,amberieu en bugey,736,730.0,01,01004_0003,01004_3,Ambérieu-en-Bugey,01,Ain


## Etude des rebuts

In [23]:
rebuts_reu = df_bv[~df_bv.id_brut_reu.isin(df_inner_matching.id_brut_reu)]
len(rebuts_reu)

2464

In [24]:
rebuts_reu.head()

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement
1543,02821_0,2821_0,02821,,bureau de vote 1,1,Rue Emptinne,02870,vivaise,323,323.0,02
2086,04079_1 - 1,4079_1 1,04079,1 1,batiment communal,,Maison des Associations et de la Culture,04160,l escale,699,689.0,04
2096,04090_1 - 1,4090_1 1,04090,1 1,hotel de ville,,LA PLACE,04240,le fugeret,122,74.0,04
2103,04095_1 - 1,4095_1 1,04095,1 1,m a i r i e,1,Le perron,04150,hospitalet,71,68.0,04
2106,04099_1 - 2,4099_1 2,04099,1 2,salle paul germain,,rue du Cimetière,04170,lambruisse,72,72.0,04


In [25]:
rebuts_miom = df_bv_miom[~df_bv_miom.id_brut_miom.isin(df_inner_matching.id_brut_miom)]
len(rebuts_miom)

3307

In [26]:
rebuts_miom.head()

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
104,01053_0020,01053_20,01053,20,Bourg-en-Bresse,01,Ain
1054,02408_0020,02408_20,02408,20,Laon,02,Aisne
1545,02821_0001,02821_1,02821,1,Vivaise,02,Aisne
1838,03190_0020,03190_20,03190,20,Moulins,03,Allier
2083,04070_0013,04070_13,04070,13,Digne-les-Bains,04,Alpes-de-Haute-Provence


## Rattraper rebuts seuls dans leur commune

In [27]:
nb_codes_communes_reu = rebuts_reu["code_commune"].value_counts()
nb_codes_communes_miom = rebuts_miom["code_commune"].value_counts()

codes_communes_bv_seuls = nb_codes_communes_reu[nb_codes_communes_reu == 1].index
codes_communes_bv_miom_seuls = nb_codes_communes_miom[nb_codes_communes_miom == 1].index

In [28]:
rebuts_reu_seuls = rebuts_reu[rebuts_reu["code_commune"].isin(codes_communes_bv_seuls)]
len(rebuts_reu_seuls)

458

In [29]:
rebuts_miom_seuls = rebuts_miom[rebuts_miom["code_commune"].isin(codes_communes_bv_miom_seuls)]
len(rebuts_miom_seuls)

748

In [30]:
rebuts_rattrapables = pd.merge(rebuts_reu_seuls, rebuts_miom_seuls,
                           on=[
                               'code_commune'
                           ],
                           how='inner')

In [31]:
print(len(rebuts_rattrapables))

rebuts_rattrapables.head()

427


,id_brut_reu,id_brut_insee,code_commune,code_x,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement,id_brut_miom,id_clean_miom,code_y,commune_miom,departement_miom,libelle_departement_miom
0,02821_0,2821_0,02821,,bureau de vote 1,1,Rue Emptinne,02870,vivaise,323,323.0,02,02821_0001,02821_1,1,Vivaise,02,Aisne
1,04079_1 - 1,4079_1 1,04079,1 1,batiment communal,,Maison des Associations et de la Culture,04160,l escale,699,689.0,04,04079_0001,04079_1,1,L'Escale,04,Alpes-de-Haute-Provence
2,04090_1 - 1,4090_1 1,04090,1 1,hotel de ville,,LA PLACE,04240,le fugeret,122,74.0,04,04090_0001,04090_1,1,Le Fugeret,04,Alpes-de-Haute-Provence
3,04095_1 - 1,4095_1 1,04095,1 1,m a i r i e,1,Le perron,04150,hospitalet,71,68.0,04,04095_0001,04095_1,1,L'Hospitalet,04,Alpes-de-Haute-Provence
4,04099_1 - 2,4099_1 2,04099,1 2,salle paul germain,,rue du Cimetière,04170,lambruisse,72,72.0,04,04099_0001,04099_1,1,Lambruisse,04,Alpes-de-Haute-Provence


In [32]:
df_matching = pd.concat([df_inner_matching, rebuts_rattrapables], axis=0).drop_duplicates()

## Etude des vrais rebuts

### Premières observations

In [33]:
vrais_rebuts_reu = rebuts_reu[~rebuts_reu.id_brut_reu.isin(rebuts_rattrapables.id_brut_reu)].sort_values('code_commune')
vrais_rebuts_reu

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement
3098,06088_533,None,06088,533,philippe seguin,6,RUE ALFRED BINET,06000,nice,65,65.0,06
3524,07045_1 - 2,7045_1 2,07045,1 2,salle des fetes,10,Rue de l'église,07450,burzet,6,6.0,07
3525,07045_1 - 3,7045_1 3,07045,1 3,salle des fetes,10,Rue de l'église,07450,burzet,345,337.0,07
3739,07230_1 - 1,7230_1 1,07230,1 1,mairie,,LE VILLAGE,07200,saint etienne de boulogne,96,70.0,07
4584,09123_,None,09123,,unique,,Mairie,09350,fornex,1,1.0,09
...,...,...,...,...,...,...,...,...,...,...,...,...
62355,90010_B-1,90010_B 1,90010,b1,ecole elementaire victor hugo,3,rue François Géant,90000,belfort,267,259.0,90
62354,90010_A-2,90010_A 2,90010,a2,salle des fetes,,Place de la République,90000,belfort,181,176.0,90
62353,90010_A-1,90010_A 1,90010,a1,hotel de ville,,Place d'Armes,90000,belfort,228,225.0,90
62365,90010_E-3,90010_E 3,90010,e3,ecole maternelle raymond aubert,19,rue de la Première Armée Française,90000,belfort,152,151.0,90


In [34]:
vrais_rebuts_miom = rebuts_miom[~rebuts_miom.id_brut_miom.isin(rebuts_rattrapables.id_brut_miom)].sort_values('code_commune')
vrais_rebuts_miom

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
104,01053_0020,01053_20,01053,20,Bourg-en-Bresse,01,Ain
1054,02408_0020,02408_20,02408,20,Laon,02,Aisne
1838,03190_0020,03190_20,03190,20,Moulins,03,Allier
2083,04070_0013,04070_13,04070,13,Digne-les-Bains,04,Alpes-de-Haute-Provence
2150,04126_0001,04126_1,04126,1,Montclar,04,Alpes-de-Haute-Provence
...,...,...,...,...,...,...,...
69677,ZZ229_0001,ZZ229_1,ZZ229,1,Zurich,ZZ,Français établis hors de France
69678,ZZ231_0001,ZZ231_1,ZZ231,1,Taipei,ZZ,Français établis hors de France
69679,ZZ233_0001,ZZ233_1,ZZ233,1,Nour-Soultan,ZZ,Français établis hors de France
69680,ZZ234_0001,ZZ234_1,ZZ234,1,Monterrey,ZZ,Français établis hors de France


In [35]:
vrais_rebuts_reu[vrais_rebuts_reu["code_commune"].apply(lambda x: x.isnumeric() and int(x[:2]) < 97)]

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement
3098,06088_533,None,06088,533,philippe seguin,6,RUE ALFRED BINET,06000,nice,65,65.0,06
3524,07045_1 - 2,7045_1 2,07045,1 2,salle des fetes,10,Rue de l'église,07450,burzet,6,6.0,07
3525,07045_1 - 3,7045_1 3,07045,1 3,salle des fetes,10,Rue de l'église,07450,burzet,345,337.0,07
3739,07230_1 - 1,7230_1 1,07230,1 1,mairie,,LE VILLAGE,07200,saint etienne de boulogne,96,70.0,07
4584,09123_,None,09123,,unique,,Mairie,09350,fornex,1,1.0,09
...,...,...,...,...,...,...,...,...,...,...,...,...
62355,90010_B-1,90010_B 1,90010,b1,ecole elementaire victor hugo,3,rue François Géant,90000,belfort,267,259.0,90
62354,90010_A-2,90010_A 2,90010,a2,salle des fetes,,Place de la République,90000,belfort,181,176.0,90
62353,90010_A-1,90010_A 1,90010,a1,hotel de ville,,Place d'Armes,90000,belfort,228,225.0,90
62365,90010_E-3,90010_E 3,90010,e3,ecole maternelle raymond aubert,19,rue de la Première Armée Française,90000,belfort,152,151.0,90


In [36]:
vrais_rebuts_miom[vrais_rebuts_miom["code_commune"].apply(lambda x: x.isnumeric() and int(x[:2]) < 97)]

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
104,01053_0020,01053_20,01053,20,Bourg-en-Bresse,01,Ain
1054,02408_0020,02408_20,02408,20,Laon,02,Aisne
1838,03190_0020,03190_20,03190,20,Moulins,03,Allier
2083,04070_0013,04070_13,04070,13,Digne-les-Bains,04,Alpes-de-Haute-Provence
2150,04126_0001,04126_1,04126,1,Montclar,04,Alpes-de-Haute-Provence
...,...,...,...,...,...,...,...
62836,91228_0034,91228_34,91228,34,Evry-Courcouronnes,91,Essonne
64173,92050_0050,92050_50,92050,50,Nanterre,92,Hauts-de-Seine
64598,93008_0023,93008_23,93008,23,Bobigny,93,Seine-Saint-Denis
65517,94028_0046,94028_46,94028,46,Créteil,94,Val-de-Marne


In [37]:
vrais_rebuts_reu[vrais_rebuts_reu["code_commune"].apply(lambda x: x.isnumeric() and int(x[:2]) >= 97)]

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement


In [38]:
vrais_rebuts_miom[vrais_rebuts_miom["code_commune"].apply(lambda x: x.isnumeric() and int(x[:2]) >= 97)]

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
66996,ZA105_0013,97105_13,97105,13,Basse-Terre,971,Guadeloupe
67411,ZB209_0061,97209_61,97209,61,Fort-de-France,972,Martinique
67681,ZC302_0033,97302_33,97302,33,Cayenne,973,Guyane
68123,ZD411_0134,97411_134,97411,134,Saint-Denis,974,La Réunion
68831,ZM611_0183,97611_183,97611,183,Mamoudzou,976,Mayotte
...,...,...,...,...,...,...,...
69436,ZP757_0005,98757_5,98757,5,Ua-Pou,987,Polynésie française
69437,ZP757_0006,98757_6,98757,6,Ua-Pou,987,Polynésie française
69438,ZP758_0001,98758_1,98758,1,Uturoa,987,Polynésie française
69439,ZP758_0002,98758_2,98758,2,Uturoa,987,Polynésie française


In [39]:
vrais_rebuts_reu[vrais_rebuts_reu["code_commune"].apply(lambda x: not(x.isnumeric()))]

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement
17292,2A130_5,None,2A130,5,bureau de vote n 5,,MAIRIE ANNEXE DE PORTICCIO,20166,grosseto prugna,432,415.0,2A
17451,2B042_5,None,2B042,5,centre administratif,,Salle du Patio,20290,borgo,724,721.0,2B
17552,2B166_1 - 1,2B166_1 1,2B166,1 1,mairie village 1,54,PLACE DE L'EGLISE,20290,monte,105,78.0,2B
17553,2B166_2 - 3,2B166_2 3,2B166,2 3,mairie annexe 2,,,20290,monte,174,161.0,2B


In [40]:
vrais_rebuts_miom[vrais_rebuts_miom["code_commune"].apply(lambda x: not(x.isnumeric()))]

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
11149,2A004_0042,2A004_42,2A004,42,Ajaccio,2A,Corse-du-Sud
11381,2B033_0027,2B033_27,2B033,27,Bastia,2B,Haute-Corse
11495,2B166_0002,2B166_2,2B166,2,Monte,2B,Haute-Corse
11494,2B166_0001,2B166_1,2B166,1,Monte,2B,Haute-Corse
68894,ZN801_0001,ZN801_1,ZN801,1,Belep,ZN,Nouvelle-Calédonie
...,...,...,...,...,...,...,...
69677,ZZ229_0001,ZZ229_1,ZZ229,1,Zurich,ZZ,Français établis hors de France
69678,ZZ231_0001,ZZ231_1,ZZ231,1,Taipei,ZZ,Français établis hors de France
69679,ZZ233_0001,ZZ233_1,ZZ233,1,Nour-Soultan,ZZ,Français établis hors de France
69680,ZZ234_0001,ZZ234_1,ZZ234,1,Monterrey,ZZ,Français établis hors de France


### Les rebuts sont-ils dans les mêmes communes d'un côté et de l'autre ?

In [41]:
vrais_rebuts_reu_dans_communes_avec_miom = vrais_rebuts_reu[vrais_rebuts_reu["code_commune"].isin(set(vrais_rebuts_miom["code_commune"]))]
vrais_rebuts_miom_dans_communes_avec_reu = vrais_rebuts_miom[vrais_rebuts_miom["code_commune"].isin(set(vrais_rebuts_reu["code_commune"]))]

In [42]:
vrais_rebuts_reu_dans_communes_avec_miom

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement
3524,07045_1 - 2,7045_1 2,07045,1 2,salle des fetes,10,Rue de l'église,07450,burzet,6,6.0,07
3525,07045_1 - 3,7045_1 3,07045,1 3,salle des fetes,10,Rue de l'église,07450,burzet,345,337.0,07
4959,10112_01 - 1,10112_01 1,10112,1 1,salle de la petite ecole,,,00000,,1,1.0,10
4960,10112_01 - 2,10112_01 2,10112,1 2,salle de la petite ecole,,,00000,,78,77.0,10
5295,10387_304,10387_304,10387,304,vassaules 2eme bureau,,139 rue Etienne Pédron,10000,troyes,428,426.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...
62355,90010_B-1,90010_B 1,90010,b1,ecole elementaire victor hugo,3,rue François Géant,90000,belfort,267,259.0,90
62354,90010_A-2,90010_A 2,90010,a2,salle des fetes,,Place de la République,90000,belfort,181,176.0,90
62353,90010_A-1,90010_A 1,90010,a1,hotel de ville,,Place d'Armes,90000,belfort,228,225.0,90
62365,90010_E-3,90010_E 3,90010,e3,ecole maternelle raymond aubert,19,rue de la Première Armée Française,90000,belfort,152,151.0,90


In [43]:
vrais_rebuts_miom_dans_communes_avec_reu

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
3529,07045_0001,07045_1,07045,1,Burzet,07,Ardèche
4964,10112_0001,10112_1,10112,1,Coussegrey,10,Aube
5300,10387_0029,10387_29,10387,29,Troyes,10,Aube
5301,10387_0001,10387_1,10387,1,Troyes,10,Aube
5302,10387_0002,10387_2,10387,2,Troyes,10,Aube
...,...,...,...,...,...,...,...
62409,90010_0009,90010_9,90010,9,Belfort,90,Territoire de Belfort
62410,90010_0010,90010_10,90010,10,Belfort,90,Territoire de Belfort
62411,90010_0011,90010_11,90010,11,Belfort,90,Territoire de Belfort
62412,90010_0012,90010_12,90010,12,Belfort,90,Territoire de Belfort


### Rebuts liés aux arrondissements

#### Premières observations

In [44]:
rebuts_reu_arrondissements = vrais_rebuts_reu[vrais_rebuts_reu["departement"].isin(["13", "69", "75"])]
len(rebuts_reu_arrondissements)

1692

In [45]:
rebuts_miom_arrondissements = vrais_rebuts_miom[vrais_rebuts_miom["departement_miom"].isin(["13", "69", "75"])]
len(rebuts_miom_arrondissements)

1695

In [46]:
rebuts_reu_communes_differentes_hors_arrondissements = vrais_rebuts_reu[~(
    vrais_rebuts_reu['id_brut_reu'].isin(set(vrais_rebuts_reu_dans_communes_avec_miom['id_brut_reu'])) |
    vrais_rebuts_reu['departement'].isin(["13", "69", "75"])
)]

len(rebuts_reu_communes_differentes_hors_arrondissements)

37

In [47]:
vrais_rebuts_miom_metropole = vrais_rebuts_miom[vrais_rebuts_miom["code_commune"].apply(lambda x: x.isnumeric() and int(x[:2]) < 97)]

rebuts_miom_communes_differentes_hors_arrondissements = vrais_rebuts_miom_metropole[~(
    vrais_rebuts_miom_metropole['id_brut_miom'].isin(set(vrais_rebuts_miom_dans_communes_avec_reu['id_brut_miom'])) |
    vrais_rebuts_miom_metropole['departement_miom'].isin(["13", "69", "75"])
)]

len(rebuts_miom_communes_differentes_hors_arrondissements)

87

#### Marseille

In [48]:
rebuts_reu_marseille = vrais_rebuts_reu[vrais_rebuts_reu["departement"] == '13']
rebuts_reu_marseille

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement
6746,13035_B05,13035_B05,13035,b05,bureau 5 salle de la gare,,Avenue de la Gare,13430,eyguieres,667,649.0,13
6745,13035_B04,13035_B04,13035,b04,bureau 4 ecole trecasteaux,,Faubourg de la Libération,13430,eyguieres,612,611.0,13
6742,13035_B01,13035_B01,13035,b01,bureau 1 moulin alcazar,,Place Thiers,13430,eyguieres,534,529.0,13
6743,13035_B02,13035_B02,13035,b02,bureau 2 ecole gilous,,Avenue de la Guillaumette,13430,eyguieres,705,699.0,13
6744,13035_B03,13035_B03,13035,b03,bureau 3 foyer du 3eme age,,Place Monier,13430,eyguieres,506,501.0,13
...,...,...,...,...,...,...,...,...,...,...,...,...
7775,13216_1608,13216_1608,13216,1608,e elem saint henri ii,14,PCE RAPHEL,13016,marseille,593,586.0,13
7774,13216_1607,13216_1607,13216,1607,e elem saint henri i,95,RUE RABELAIS,13016,marseille,411,402.0,13
7773,13216_1606,13216_1606,13216,1606,e elem saint henri i,95,RUE RABELAIS,13016,marseille,393,382.0,13
7772,13216_1602,13216_1602,13216,1602,e elem saint andre condorcet,ENTREE 430,BD HENRI BARNIER,13016,marseille,598,586.0,13


In [49]:
rebuts_miom_marseille = vrais_rebuts_miom[vrais_rebuts_miom["departement_miom"] == '13']
rebuts_miom_marseille

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
6749,13035_0001,13035_1,13035,1,Eyguières,13,Bouches-du-Rhône
6750,13035_0002,13035_2,13035,2,Eyguières,13,Bouches-du-Rhône
6751,13035_0003,13035_3,13035,3,Eyguières,13,Bouches-du-Rhône
6752,13035_0004,13035_4,13035,4,Eyguières,13,Bouches-du-Rhône
6753,13035_0005,13035_5,13035,5,Eyguières,13,Bouches-du-Rhône
...,...,...,...,...,...,...,...
7088,13055_1343,13055_1343,13055,1343,Marseille,13,Bouches-du-Rhône
7440,13061_0001,13061_1,13061,1,Saint-Pierre-de-Mézoargues,13,Bouches-du-Rhône
7650,13102_0001,13102_1,13102,1,Saint-Victoret,13,Bouches-du-Rhône
7651,13102_0002,13102_2,13102,2,Saint-Victoret,13,Bouches-du-Rhône


In [50]:
rebuts_reu_marseille['code_commune'] = rebuts_reu_marseille['code_commune'].apply(
    lambda x: '13055' if x[2] == '2' else x
)

rebuts_reu_marseille[rebuts_reu_marseille[['code_commune', 'code']].duplicated(keep=False)]

/tmp/ipykernel_775/2554710340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rebuts_reu_marseille['code_commune'] = rebuts_reu_marseille['code_commune'].apply(


,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement


In [51]:
rattrapage_rebuts_marseille = pd.merge(rebuts_reu_marseille, rebuts_miom_marseille,
                           on=[
                               'code', 'code_commune'
                           ],
                           how='inner')

len(rattrapage_rebuts_marseille)

480

#### Lyon

In [52]:
rebuts_reu_lyon = vrais_rebuts_reu[vrais_rebuts_reu["departement"] == '69']
rebuts_reu_lyon

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement
48263,69381_113,69381_113,69381,113,ecole robert doisneau,1,rue Sergent Blandan,69001,lyon,227,226.0,69
48262,69381_112,69381_112,69381,112,ecole robert doisneau,1,rue Sergent Blandan,69001,lyon,297,296.0,69
48264,69381_114,69381_114,69381,114,ecole robert doisneau,1,rue Sergent Blandan,69001,lyon,317,309.0,69
48266,69381_116,69381_116,69381,116,groupe scolaire raoul dufy,4,rue Vaucanson,69001,lyon,214,212.0,69
48267,69381_117,69381_117,69381,117,ecole tables claudiennes,2,rue des Tables Claudiennes,69001,lyon,228,227.0,69
...,...,...,...,...,...,...,...,...,...,...,...,...
48527,69389_903,69389_903,69389,903,mairie du 9 eme,6,place du Marché,69009,lyon,314,280.0,69
48526,69389_902,69389_902,69389,902,mairie du 9 eme,6,place du Marché,69009,lyon,180,180.0,69
48525,69389_901,69389_901,69389,901,mairie du 9 eme,6,place du Marché,69009,lyon,328,325.0,69
48539,69389_915,69389_915,69389,915,gymnase audrey hepburn,5,rue Laure Diebold,69009,lyon,246,236.0,69


In [53]:
rebuts_miom_lyon = vrais_rebuts_miom[vrais_rebuts_miom["departement_miom"] == '69']
rebuts_miom_lyon

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
47820,69123_0820,69123_820,69123,820,Lyon,69,Rhône
47819,69123_0810,69123_810,69123,810,Lyon,69,Rhône
47816,69123_0807,69123_807,69123,807,Lyon,69,Rhône
47818,69123_0809,69123_809,69123,809,Lyon,69,Rhône
47817,69123_0808,69123_808,69123,808,Lyon,69,Rhône
...,...,...,...,...,...,...,...
47715,69123_0215,69123_215,69123,215,Lyon,69,Rhône
47714,69123_0210,69123_210,69123,210,Lyon,69,Rhône
47713,69123_0209,69123_209,69123,209,Lyon,69,Rhône
47712,69123_0208,69123_208,69123,208,Lyon,69,Rhône


In [54]:
rebuts_reu_lyon['code_commune'] = rebuts_reu_lyon['code_commune'].apply(
    lambda x: '69123' if x[2] == '3' else x
)

rebuts_reu_lyon[rebuts_reu_lyon[['code_commune', 'code']].duplicated(keep=False)]

/tmp/ipykernel_775/886085551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rebuts_reu_lyon['code_commune'] = rebuts_reu_lyon['code_commune'].apply(


,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement


In [55]:
rattrapage_rebuts_lyon = pd.merge(rebuts_reu_lyon, rebuts_miom_lyon,
                           on=[
                               'code', 'code_commune'
                           ],
                           how='inner')

len(rattrapage_rebuts_lyon)

304

#### Paris

In [56]:
rebuts_reu_paris = vrais_rebuts_reu[vrais_rebuts_reu["departement"] == '75']
rebuts_reu_paris

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement
51646,75101_1,75101_1,75101,1,ex mairie du 1er arrondissement,4,PLACE DU LOUVRE,75001,paris,372,366.0,75
51647,75101_10,75101_10,75101,10,ecole elementaire,28,RUE CAMBON,75001,paris,309,307.0,75
51648,75101_2,75101_2,75101,2,ecole elementaire,19,RUE DE L'ARBRE SEC,75001,paris,448,448.0,75
51649,75101_3,75101_3,75101,3,ecole maternelle,6,RUE SAINT-GERMAIN L'AUXERROIS,75001,paris,472,466.0,75
51650,75101_4,75101_4,75101,4,mediatheque de la canopee la fontaine,10,PASSAGE DE LA CANOPEE,75001,paris,382,377.0,75
...,...,...,...,...,...,...,...,...,...,...,...,...
52492,75120_31,75120_31,75120,31,ecole elementaire,20,RUE LE VAU,75020,paris,502,501.0,75
52491,75120_30,75120_30,75120,30,college pierre mendes france,24,RUE LE VAU,75020,paris,419,416.0,75
52490,75120_3,75120_3,75120,3,etablissement scolaire saint germain de charonne,3,RUE DES PRAIRIES,75020,paris,331,327.0,75
52489,75120_29,75120_29,75120,29,ecole elementaire,4,RUE PIERRE FONCIN,75020,paris,370,172.0,75


In [57]:
rebuts_miom_paris = vrais_rebuts_miom[vrais_rebuts_miom["departement_miom"] == '75']
rebuts_miom_paris

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
52289,75056_1534,75056_1534,75056,1534,Paris,75,Paris
52288,75056_1533,75056_1533,75056,1533,Paris,75,Paris
52287,75056_1532,75056_1532,75056,1532,Paris,75,Paris
52286,75056_1531,75056_1531,75056,1531,Paris,75,Paris
52290,75056_1535,75056_1535,75056,1535,Paris,75,Paris
...,...,...,...,...,...,...,...
51985,75056_2062,75056_2062,75056,2062,Paris,75,Paris
51984,75056_2061,75056_2061,75056,2061,Paris,75,Paris
51983,75056_2060,75056_2060,75056,2060,Paris,75,Paris
51982,75056_2059,75056_2059,75056,2059,Paris,75,Paris


In [58]:
rebuts_reu_paris['code'] = rebuts_reu_paris['code'].apply(lambda x: "0" + x)
rebuts_reu_paris['code'] = rebuts_reu_paris['code_commune'].apply(lambda x: x[-2:]) + rebuts_reu_paris['code'].apply(lambda x: x[-2:])
rebuts_reu_paris['code'] = rebuts_reu_paris['code'].apply(lambda x: x[1:] if x[0] == '0' else x)

rebuts_reu_paris['code_commune'] = "75056"

rebuts_reu_paris[rebuts_reu_paris[['code_commune', 'code']].duplicated(keep=False)]

/tmp/ipykernel_775/3853486837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rebuts_reu_paris['code'] = rebuts_reu_paris['code'].apply(lambda x: "0" + x)
/tmp/ipykernel_775/3853486837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rebuts_reu_paris['code'] = rebuts_reu_paris['code_commune'].apply(lambda x: x[-2:]) + rebuts_reu_paris['code'].apply(lambda x: x[-2:])
/tmp/ipykernel_775/3853486837.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,departement


In [59]:
rattrapage_rebuts_paris = pd.merge(rebuts_reu_paris, rebuts_miom_paris,
                           on=[
                               'code', 'code_commune'
                           ],
                           how='inner')

len(rattrapage_rebuts_paris)

859

#### L'heure des résultats

In [60]:
df_matching = pd.concat([df_matching, rattrapage_rebuts_marseille, rattrapage_rebuts_lyon, rattrapage_rebuts_paris], axis=0).drop_duplicates()

In [61]:
len(df_matching)

68445

In [62]:
vrais_rebuts_arrondissements_reu = rebuts_reu_arrondissements[
    ~rebuts_reu_arrondissements.id_brut_reu.isin(df_matching.id_brut_reu)
].sort_values('code_commune')

len(vrais_rebuts_arrondissements_reu)

49

In [63]:
vrais_rebuts_arrondissements_miom = rebuts_miom_arrondissements[
    ~rebuts_miom_arrondissements.id_brut_miom.isin(df_matching.id_brut_miom)
].sort_values('code_commune')

len(vrais_rebuts_arrondissements_miom)

52

## Fin et matching manuel

### Matching réussi

In [64]:
df_correspondance = df_matching[['id_brut_reu', 'id_brut_miom']]
len(df_correspondance)

68445

In [65]:
df_bv_avec_correspondance = pd.merge(df_bv_originale, df_correspondance,
                           on=[
                               'id_brut_reu'
                           ],
                           how='inner')

len(df_bv_avec_correspondance)

68445

In [66]:
df_bv_avec_correspondance.head()

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,id_brut_miom
0,01001_1,1001_1,01001,1,bureau 1,,Salle des fêtes,01400,abergement clemenciat,448,448.0,01001_0001
1,01002_1,1002_1,01002,1,mairie,1,Place de la Mairie,01640,l abergement de varey,157,143.0,01002_0001
2,01004_1,1004_1,01004,1,b1 espace 1500,,AVENUE LEON BLUM,01500,amberieu en bugey,633,630.0,01004_0001
3,01004_2,1004_2,01004,2,b2 espace 1500,,AVENUE LEON BLUM,01500,amberieu en bugey,640,638.0,01004_0002
4,01004_3,1004_3,01004,3,b3 chateau des echelles,,RUE DES ARENES,01500,amberieu en bugey,736,730.0,01004_0003


In [67]:
# df_bv_avec_correspondance.to_parquet("table_bv_avec_correspondances_miom.parquet", index=False)

### Matching raté

In [68]:
matching_rate_reu = df_bv_originale[~df_bv_originale.id_brut_reu.isin(df_correspondance.id_brut_reu)].sort_values('id_brut_reu')

matching_rate_miom = df_bv_miom[~df_bv_miom.id_brut_miom.isin(df_correspondance.id_brut_miom)].sort_values('id_brut_miom')
matching_rate_miom = matching_rate_miom[matching_rate_miom['departement_miom'].apply(lambda x: x.isnumeric())]

In [69]:
matching_rate_reu

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final
3098,06088_533,None,06088,533,philippe seguin,6,RUE ALFRED BINET,06000,nice,65,65.0
3524,07045_1 - 2,7045_1 2,07045,1 2,salle des fetes,10,Rue de l'église,07450,burzet,6,6.0
3525,07045_1 - 3,7045_1 3,07045,1 3,salle des fetes,10,Rue de l'église,07450,burzet,345,337.0
3739,07230_1 - 1,7230_1 1,07230,1 1,mairie,,LE VILLAGE,07200,saint etienne de boulogne,96,70.0
4584,09123_,None,09123,,unique,,Mairie,09350,fornex,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
62376,90010_L-2,90010_L 2,90010,l 2,ecole maternelle des barres,9,Via d'Auxelles,90000,belfort,276,274.0
62377,90010_L-3,90010_L 3,90010,l 3,ecole elementaire des barres,10,rue Jules Siegfried,90000,belfort,278,278.0
62378,90010_M-1,90010_M 1,90010,m 1,ecole maternelle antoine de saint exupery,,Rue de la Paix,90000,belfort,183,183.0
62379,90010_N-1,90010_N 1,90010,n 1,gymnase serzian,,Rue Floréal,90000,belfort,260,259.0


In [70]:
matching_rate_miom

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
104,01053_0020,01053_20,01053,20,Bourg-en-Bresse,01,Ain
1054,02408_0020,02408_20,02408,20,Laon,02,Aisne
1838,03190_0020,03190_20,03190,20,Moulins,03,Allier
2083,04070_0013,04070_13,04070,13,Digne-les-Bains,04,Alpes-de-Haute-Provence
2150,04126_0001,04126_1,04126,1,Montclar,04,Alpes-de-Haute-Provence
...,...,...,...,...,...,...,...
69437,ZP757_0006,98757_6,98757,6,Ua-Pou,987,Polynésie française
69438,ZP758_0001,98758_1,98758,1,Uturoa,987,Polynésie française
69439,ZP758_0002,98758_2,98758,2,Uturoa,987,Polynésie française
69440,ZP758_0003,98758_3,98758,3,Uturoa,987,Polynésie française


### Matching à la main

In [71]:
matching_a_la_main_reu = matching_rate_reu[matching_rate_reu["code_commune"].isin(set(matching_rate_miom["code_commune"]))]
matching_a_la_main_miom = matching_rate_miom[matching_rate_miom["code_commune"].isin(set(matching_rate_reu["code_commune"]))]

In [72]:
matching_a_la_main_reu

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final
3524,07045_1 - 2,7045_1 2,07045,1 2,salle des fetes,10,Rue de l'église,07450,burzet,6,6.0
3525,07045_1 - 3,7045_1 3,07045,1 3,salle des fetes,10,Rue de l'église,07450,burzet,345,337.0
4959,10112_01 - 1,10112_01 1,10112,1 1,salle de la petite ecole,,,00000,,1,1.0
4960,10112_01 - 2,10112_01 2,10112,1 2,salle de la petite ecole,,,00000,,78,77.0
5279,10387_101,10387_101,10387,101,hotel de ville 1er bureau,,place Alexandre Israël,10000,troyes,296,295.0
...,...,...,...,...,...,...,...,...,...,...,...
62376,90010_L-2,90010_L 2,90010,l 2,ecole maternelle des barres,9,Via d'Auxelles,90000,belfort,276,274.0
62377,90010_L-3,90010_L 3,90010,l 3,ecole elementaire des barres,10,rue Jules Siegfried,90000,belfort,278,278.0
62378,90010_M-1,90010_M 1,90010,m 1,ecole maternelle antoine de saint exupery,,Rue de la Paix,90000,belfort,183,183.0
62379,90010_N-1,90010_N 1,90010,n 1,gymnase serzian,,Rue Floréal,90000,belfort,260,259.0


In [73]:
matching_a_la_main_miom

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
3529,07045_0001,07045_1,07045,1,Burzet,07,Ardèche
4964,10112_0001,10112_1,10112,1,Coussegrey,10,Aube
5301,10387_0001,10387_1,10387,1,Troyes,10,Aube
5302,10387_0002,10387_2,10387,2,Troyes,10,Aube
5303,10387_0003,10387_3,10387,3,Troyes,10,Aube
...,...,...,...,...,...,...,...
62426,90010_0025,90010_25,90010,25,Belfort,90,Territoire de Belfort
62427,90010_0026,90010_26,90010,26,Belfort,90,Territoire de Belfort
62428,90010_0027,90010_27,90010,27,Belfort,90,Territoire de Belfort
62429,90010_0028,90010_28,90010,28,Belfort,90,Territoire de Belfort


In [74]:
matching_a_la_main_reu.to_csv("matching_a_faire_a_la_main_reu.csv", index=False)
matching_a_la_main_miom.to_csv("matching_a_faire_a_la_main_miom.csv", index=False)

A partir des deux CSV générés (environ 300 lignes chacun), nous avons effectué un appariement à la main entre les deux référentiels lorsque les paires étaient évidentes. Cela concerne notamment :

- Des bureaux de vote découpés en 2 d'un côté mais pas de l'autre (ex: "bureau 1" d'un côté et "bureau 1 1" et "bureau 1 2" de l'autre)
- Des bureaux de vote pour lesquels un caractère précède les numéros de bureaux d'un côté mais pas de l'autre, les caractères observés étant entre autres "a", "b", "1", "10"

Un CSV est alors retourné avec les identifiants des paires retrouvées dans les deux référentiels. Dans la pratique, près de la moitié des bv du REU envoyés en matching manuel ont pu être attribués à un bureau du MIOM.

In [75]:
os.system(f"mc cp s3/projet-adresses-reu/table_correspondance_a_la_main.csv table_correspondance_a_la_main.csv")

In [76]:
table_correspondance_a_la_main = pd.read_csv('table_correspondance_a_la_main.csv')

matching_fait_a_la_main = pd.merge(df_bv_originale, table_correspondance_a_la_main,
                           on=['id_brut_reu'],
                           how='inner')

len(matching_fait_a_la_main)

270

### Fin et sauvegarde

In [77]:
df_bv_finale_inner = pd.concat([df_bv_avec_correspondance, matching_fait_a_la_main], axis=0).drop_duplicates()
df_bv_finale_outer = pd.merge(df_bv_originale, df_bv_finale_inner,
                              how='left')

print(len(df_bv_finale_inner))
print(len(df_bv_finale_outer))

68715
68839


In [78]:
df_bv_finale_outer.head(3)

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final,id_brut_miom
0,01001_1,1001_1,01001,1,bureau 1,,Salle des fêtes,01400,abergement clemenciat,448,448.0,01001_0001
1,01002_1,1002_1,01002,1,mairie,1,Place de la Mairie,01640,l abergement de varey,157,143.0,01002_0001
2,01004_1,1004_1,01004,1,b1 espace 1500,,AVENUE LEON BLUM,01500,amberieu en bugey,633,630.0,01004_0001


In [79]:
df_bv_finale_outer.rename(columns={"id_brut_reu": "id_brut_bv_reu"}, inplace=True)
df_bv_finale_outer.to_parquet("table_bv_finale.parquet", index=False)

In [80]:
os.system(f"mc cp table_bv_finale.parquet s3/projet-adresses-reu/table_bv_finale.parquet")

In [81]:
rebuts_finaux_reu = df_bv_originale[~df_bv_originale.id_brut_reu.isin(df_bv_finale_inner.id_brut_reu)].sort_values('id_brut_reu')

rebuts_finaux_miom = df_bv_miom[~df_bv_miom.id_brut_miom.isin(df_bv_finale_inner.id_brut_miom)].sort_values('id_brut_miom')
rebuts_finaux_miom = rebuts_finaux_miom[rebuts_finaux_miom['departement_miom'].apply(lambda x: x.isnumeric() and x[:2] != '98')]
# rebuts_finaux_miom = rebuts_finaux_miom[rebuts_finaux_miom["code_commune"].isin(set(rebuts_finaux_reu["code_commune"]))]

In [82]:
rebuts_finaux_reu.to_parquet("rebuts_reu.parquet", index=False)
rebuts_finaux_reu

,id_brut_reu,id_brut_insee,code_commune,code,libelle_reu,num_voie_reu,voie_reu,cp_reu,commune_reu,nb_adresses_initial,nb_adresses_final
3098,06088_533,None,06088,533,philippe seguin,6,RUE ALFRED BINET,06000,nice,65,65.0
3739,07230_1 - 1,7230_1 1,07230,1 1,mairie,,LE VILLAGE,07200,saint etienne de boulogne,96,70.0
4584,09123_,None,09123,,unique,,Mairie,09350,fornex,1,1.0
5279,10387_101,10387_101,10387,101,hotel de ville 1er bureau,,place Alexandre Israël,10000,troyes,296,295.0
5280,10387_102,10387_102,10387,102,hotel de ville 2eme bureau,,place Alexandre Israël,10000,troyes,306,301.0
...,...,...,...,...,...,...,...,...,...,...,...
62376,90010_L-2,90010_L 2,90010,l 2,ecole maternelle des barres,9,Via d'Auxelles,90000,belfort,276,274.0
62377,90010_L-3,90010_L 3,90010,l 3,ecole elementaire des barres,10,rue Jules Siegfried,90000,belfort,278,278.0
62378,90010_M-1,90010_M 1,90010,m 1,ecole maternelle antoine de saint exupery,,Rue de la Paix,90000,belfort,183,183.0
62379,90010_N-1,90010_N 1,90010,n 1,gymnase serzian,,Rue Floréal,90000,belfort,260,259.0


In [83]:
rebuts_finaux_miom.to_parquet("rebuts_miom.parquet", index=False)
rebuts_finaux_miom

,id_brut_miom,id_clean_miom,code_commune,code,commune_miom,departement_miom,libelle_departement_miom
104,01053_0020,01053_20,01053,20,Bourg-en-Bresse,01,Ain
1054,02408_0020,02408_20,02408,20,Laon,02,Aisne
1838,03190_0020,03190_20,03190,20,Moulins,03,Allier
2083,04070_0013,04070_13,04070,13,Digne-les-Bains,04,Alpes-de-Haute-Provence
2150,04126_0001,04126_1,04126,1,Montclar,04,Alpes-de-Haute-Provence
...,...,...,...,...,...,...,...
66996,ZA105_0013,97105_13,97105,13,Basse-Terre,971,Guadeloupe
67411,ZB209_0061,97209_61,97209,61,Fort-de-France,972,Martinique
67681,ZC302_0033,97302_33,97302,33,Cayenne,973,Guyane
68123,ZD411_0134,97411_134,97411,134,Saint-Denis,974,La Réunion
